In [1]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import EEGDataset
from torch.utils.data import random_split

C:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Aksel\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# load in the dataset

raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'

ds1 = EEGDataset(raw_data_dir,4, 250) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment

//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\artefact_annotations.npy
Lables for night 0 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\EEG_raw_250hz.npy
Night 0 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\artefact_annotations.npy
Lables for night 1 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\EEG_raw_250hz.npy
Night 1 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_3\artefact_annotations.npy
Lables for night 2 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_3\EEG_raw_250hz.npy
Night 2 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_

In [3]:
#Calculate class imbalance
balancing_dataloader = DataLoader(ds1, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
artefacts = 0
good_samples = 0
for batch, (X, y) in enumerate(balancing_dataloader):
    if y == 1:
        artefacts += 1
    else:
        good_samples += 1
    if batch > 100000:
        break

class_ratio = good_samples/artefacts
print("debug")


debug


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(), # Todo: Conv
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [5]:
learning_rate = 1e-2
batch_size = 64
epochs = 5

model = NeuralNetwork()

#Weighting artefacts 10x
#loss = nn.BCELoss(weight = torch.tensor([0.1,0.9]))

#weights = [1,10]
#class_weights = torch.FloatTensor(weights)#.cuda()

#loss = nn.CrossEntropyLoss(weight=class_weights) # Using cross entropy and not binary cross entropy because weight implementation is better :/
#loss = nn.CrossEntropyLoss(class_weights)

#pos_weight: amount of positive examples compared to negative examples. Calculate as: negative_examples/positive_examples
loss = nn.BCEWithLogitsLoss(pos_weight = class_ratio*torch.ones([batch_size])) 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X).reshape(-1)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        yFloat = y.type(torch.FloatTensor)

        try:
            loss = loss_fn(pred, yFloat)
        except:
            print("loss function failed")
            break


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print("Predicted values:")
            print(pred)
            print("Actual values:")
            print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            pred = model(X).reshape(-1) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")


In [7]:
# Split data into train and test data
trainSamples = int(ds1.__len__()*0.7)
testSamples = int(ds1.__len__() - trainSamples)
training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss

In [8]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.728351  [    0/1906271]
Predicted values:
tensor([0.5061, 0.5084, 0.5062, 0.5071, 0.5078, 0.5186, 0.5077, 0.5059, 0.5011,
        0.5077, 0.5088, 0.5069, 0.5055, 0.5056, 0.5032, 0.5052, 0.5053, 0.5177,
        0.5063, 0.5098, 0.5066, 0.5055, 0.5052, 0.5092, 0.5053, 0.5082, 0.5057,
        0.5047, 0.5068, 0.5063, 0.5043, 0.5051, 0.5049, 0.5144, 0.5171, 0.5068,
        0.5081, 0.5048, 0.5059, 0.5056, 0.5043, 0.5065, 0.5063, 0.5167, 0.5059,
        0.5114, 0.5294, 0.5073, 0.5036, 0.5056, 0.5078, 0.5182, 0.5144, 0.5081,
        0.5056, 0.5059, 0.5062, 0.5057, 0.5043, 0.5043, 0.5058, 0.5061, 0.5074,
        0.5051], grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
        1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0

In [9]:
print('debug')
ds1[5]
ds1[6]

debug


(array([3.08823340e+07, 4.32624756e+03, 1.18045825e+03, 1.06878003e+03,
        1.08247266e+03, 1.04963074e+03, 1.20077100e+03, 7.64120789e+02,
        1.03086731e+03, 3.41896240e+02, 3.20337097e+02, 3.67985504e+02,
        1.80468414e+02, 8.47293549e+01, 4.20016418e+02, 5.85530167e+01,
        3.37170044e+02, 1.32419220e+02, 2.59060364e+02, 2.00780731e+02,
        1.07623146e+02, 1.16960457e+02, 1.96671082e+02, 6.02180443e+01,
        3.30449310e+02, 1.53158295e+02, 2.51940552e+02, 5.34502792e+01,
        4.44208488e+01, 3.39242310e+02, 8.62816620e+01, 4.41908798e+01,
        4.16126709e+01, 8.94659119e+01, 1.68497208e+02, 1.52460129e+02,
        1.06572884e+02, 1.43524704e+02, 8.73667145e+01, 1.46133484e+02,
        6.68495178e+01, 1.25502556e+02, 1.53061646e+02, 7.16261292e+01,
        7.13538361e+01, 1.09969246e+02, 1.10196335e+02, 9.74286804e+01,
        9.19895248e+01, 5.84041939e+01, 4.36080688e+02, 1.22264832e+02,
        1.62358810e+02, 7.19400558e+01, 7.40001831e+01, 6.267993